# 네이버 영화평(알라딘)에 나온 형태소 SQLite DB에 저장
- 영화평에 나온 형태소 추출
- Okt 형태소 분석
- 형태소, 품사, 나온횟수 DB에 저장하기

In [1]:
import numpy as np
import pandas as pd
import sqlite3

In [4]:
df = pd.read_csv('../web crawling/movie_review_data.csv')
df.drop(df.columns[0], axis='columns', inplace=True)
df.head()

,score,text,date
0,40.0,맙소사 두종족이 그렇게 많이 다치고 죽었는데 그날 결혼식을 하겠다고? 장례식이 아니...,2019.10.17 14:00
1,100.0,well....well...well,2019.10.17 10:24
2,10.0,결말은 개연성이 최악입니다.인간족 여왕 때문에 요정 및 말레피센트 동족들이 수십명 ...,2019.10.17 17:07
3,40.0,오로라 때문에 몇이 고생하는 거냐?,2019.10.17 11:36
4,100.0,시사로 봤는데 안젤리나 졸리 역시는 역시다! 비주얼더 디즈니에서 엄청 공들 거 같고...,2019.10.17 09:11


In [5]:
reviews = df['text']
reviews.shape

(2666,)

In [6]:
conn = sqlite3.connect('./test.db') 
cur = conn.cursor()

In [7]:
cur = conn.cursor()
cur.execute('CREATE TABLE IF NOT EXISTS Movie_reviews \
    (word TEXT, \
     pos TEXT, \
     count INT, \
     PRIMARY KEY(word, pos));')

In [8]:
from konlpy.tag import Okt
okt = Okt()

C:\Users\EZEN\Anaconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [9]:
morphs = okt.pos(str(reviews[0]))
morphs

[('맙소사', 'Exclamation'),
 ('두', 'Determiner'),
 ('종족', 'Noun'),
 ('이', 'Josa'),
 ('그렇게', 'Adverb'),
 ('많이', 'Adverb'),
 ('다치고', 'Verb'),
 ('죽었는데', 'Verb'),
 ('그날', 'Noun'),
 ('결혼식', 'Noun'),
 ('을', 'Josa'),
 ('하겠다고', 'Verb'),
 ('?', 'Punctuation'),
 ('장례식', 'Noun'),
 ('이', 'Josa'),
 ('아니라', 'Adjective'),
 ('?', 'Punctuation'),
 ('이건', 'Noun'),
 ('조커', 'Noun'),
 ('도', 'Josa'),
 ('안', 'VerbPrefix'),
 ('할', 'Verb'),
 ('짓', 'Noun'),
 ('인데', 'Josa'),
 ('...', 'Punctuation'),
 ('감독', 'Noun'),
 ('이랑', 'Josa'),
 ('작가', 'Noun'),
 ('는', 'Josa'),
 ('반성', 'Noun'),
 ('좀', 'Noun'),
 ('해', 'Noun'),
 ('라', 'Josa'),
 ('.', 'Punctuation'),
 ('그래도', 'Adverb'),
 ('미셸', 'Noun'),
 ('파이퍼의', 'Verb'),
 ('연기', 'Noun'),
 ('와', 'Josa'),
 ('안젤리나', 'Noun'),
 ('졸리', 'Noun'),
 ('의', 'Josa'),
 ('비주', 'Noun'),
 ('얼', 'Noun'),
 ('만큼은', 'Josa'),
 ('명불허전', 'Noun'),
 ('이었다', 'Verb'),
 ('...', 'Punctuation')]

In [10]:
# Test용 데이터 입력
cur.execute("INSERT INTO Movie_reviews VALUES('윌', 'Noun', 1);")
conn.commit()

In [11]:
# Test용 데이터 조회
sql = "SELECT count FROM Movie_reviews WHERE word like ? and pos like ?;"
cur.execute(sql, ('윌', 'Noun'))
count, = cur.fetchone()
print(count)

1


In [12]:
# Test용 데이터 삭제
cur.execute("DELETE FROM Movie_reviews WHERE word like '윌';")
conn.commit()

In [13]:
def countMorph(word, pos):    
    count = 0
    try:
        cur = conn.cursor()
        sql = "SELECT count FROM Movie_reviews WHERE word like ? and pos like ?"
        cur.execute(sql, (word, pos))
        count, = cur.fetchone()
    except TypeError as te:
        count = 0
    except Exception as e: 
        print('예외가 발생했습니다.', e)
        count = -1
    finally:
        return count

In [14]:
print(countMorph('졸리', 'Noun'))
print(countMorph('안젤리나', 'Noun'))
x = ('안젤리나', 'Noun')
countMorph(*x)

0
0


0

In [15]:
def insertMorph(word, pos, count):
    cur = conn.cursor()
    sql = "INSERT INTO Movie_reviews VALUES(?, ?, ?);"
    cur.execute(sql, (word, pos, count))
    conn.commit()

In [16]:
insertMorph(*x, 1)

In [17]:
def updateMorph(word, pos, count):
    cur = conn.cursor()
    sql = "UPDATE Movie_reviews SET count=? WHERE word like ? and pos like ?;"
    cur.execute(sql, (count, word, pos))
    conn.commit()

In [18]:
updateMorph(*x, 2)

In [ ]:
i = 1
for review in reviews:
    if i % 10 == 0:
        print(i, end=' ')
    i += 1
    morphs = okt.pos(str(review))
    for morph in morphs:
        if morph[1] in ["Josa", "Eomi", "Punctuation"]:
            continue
        count = countMorph(*morph)     # Unpacking
        if count == 0:                 # 신규 등록
            insertMorph(*morph, 1)
        elif count > 0:                # count update
            updateMorph(*morph, count+1)
        else:
            print('DB Error!!!')

10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 500 510 520 530 540 550 560 570 580 590 600 610 620 630 640 650 660 670 680 690 700 710 720 730 740 750 760 770 780 790 800 810 820 830 840 850 860 870 880 890 900 910 920 930 940 950 960 970 980 990 1000 1010 1020 1030 1040 1050 1060 1070 1080 1090 1100 1110 1120 1130 1140 1150 1160 1170 1180 1190 1200 1210 1220 1230 1240 1250 1260 1270 1280 1290 1300 1310 1320 1330 1340 1350 1360 1370 1380 1390 1400 1410 1420 1430 1440 1450 1460 1470 1480 1490 1500 1510 1520 1530 1540 1550 1560 1570 1580 1590 1600 1610 1620 1630 1640 1650 1660 1670 1680 1690 1700 1710 1720 1730 1740 1750 1760 1770 1780 1790 1800 1810 1820 1830 1840 1850 1860 1870 1880 1890 1900 1910 1920 1930 1940 1950 1960 1970 1980 1990 2000 2010 

In [31]:
cur = conn.cursor()
sql = "SELECT count(*) FROM Movie_reviews;"
cur.execute(sql)
count, = cur.fetchone()
print(count)

23977


In [32]:
cur = conn.cursor()
sql = "SELECT * FROM Movie_reviews ORDER BY count DESC LIMIT 10;"
cur.execute(sql)
rows = cur.fetchall()
for row in rows:
    print(row)

('영화', 'Noun', 9609)
('너무', 'Adverb', 7616)
('노래', 'Noun', 3302)
('진짜', 'Noun', 3276)
('최고', 'Noun', 3273)
('들', 'Suffix', 3063)
('알라딘', 'Noun', 2897)
('지니', 'Noun', 2404)
('윌스미스', 'Noun', 2320)
('디즈니', 'Noun', 2246)


In [33]:
sql = "SELECT * FROM Movie_reviews;"
cur = conn.cursor()
cur.execute(sql)
rows = cur.fetchall();
columnName = ['형태소', '품사', '등장횟수']
df = pd.DataFrame(columns = columnName)
for row in rows:
    df = df.append(pd.DataFrame([list(row)], columns = columnName), 
                   ignore_index=True)
df.head()

,형태소,품사,등장횟수
0,윌,Noun,704
1,스미스,Noun,810
2,하드캐리,Noun,405
3,하여,Verb,33
4,의도,Noun,18


In [48]:
conn.close()

In [35]:
setMorphs = set(df['형태소'])
len(setMorphs)

23677

In [42]:
# 같은 형태소, 다른 품사
length = len(df)
zeros = np.zeros(length, dtype = np.int32)
df['동일형태소'] = zeros.T

In [39]:
df.head()

,형태소,품사,등장횟수,동일형태소
0,윌,Noun,704,0
1,스미스,Noun,810,0
2,하드캐리,Noun,405,0
3,하여,Verb,33,0
4,의도,Noun,18,0


In [44]:
for i in range(length-1):
    if i % 100 == 0:
        print(i, end=' ')
    if df.iloc[i,3] == 1:
        continue;
    for k in range(i+1, length):
        if df.iloc[i,0] == df.iloc[k,0]:
            df.iloc[i,3] = 1
            df.iloc[k,3] = 1
            continue

0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 3200 3300 3400 3500 3600 3700 3800 3900 4000 4100 4200 4300 4400 4500 4600 4700 4800 4900 5000 5100 5200 5300 5400 5500 5600 5700 5800 5900 6000 6100 6200 6300 6400 6500 6600 6700 6800 6900 7000 7100 7200 7300 7400 7500 7600 7700 7800 7900 8000 8100 8200 8300 8400 8500 8600 8700 8800 8900 9000 9100 9200 9300 9400 9500 9600 9700 9800 9900 10000 10100 10200 10300 10400 10500 10600 10700 10800 10900 11000 11100 11200 11300 11400 11500 11600 11700 11800 11900 12000 12100 12200 12300 12400 12500 12600 12700 12800 12900 13000 13100 13200 13300 13400 13500 13600 13700 13800 13900 14000 14100 14200 14300 14400 14500 14600 14700 14800 14900 15000 15100 15200 15300 15400 15500 15600 15700 15800 15900 16000 16100 16200 16300 16400 16500 16600 16700 16800 16900 17000 17100 17200 17300 17400 17500 17600 17700 17800 17900 18000 18100 18200 18300 18400 18

In [45]:
ddf = df[df['동일형태소'] == 1]

In [46]:
ddf

,형태소,품사,등장횟수,동일형태소
10,영화,Noun,9609,1
14,도,Noun,648,1
19,중,Suffix,670,1
28,울,Modifier,62,1
36,아,Exclamation,288,1
...,...,...,...,...
22671,이었다면,Foreign,1,1
22871,넘은게,Foreign,1,1
22882,그만,Modifier,2,1
23191,짓,VerbPrefix,2,1


In [47]:
ddf[ddf['형태소'] == '영화']

,형태소,품사,등장횟수,동일형태소
10,영화,Noun,9609,1
15265,영화,Foreign,1,1


In [50]:
df.to_csv('data/03.영화평 형태소.csv', encoding='euc-kr')

### 정렬한 후에 찾는 방법

In [12]:
df = pd.read_csv('data/03.영화평 형태소.csv', encoding='euc-kr')
df.drop(df.columns[0], axis='columns', inplace=True)
df.head()

,형태소,품사,등장횟수,동일형태소
0,윌,Noun,704,0
1,스미스,Noun,810,0
2,하드캐리,Noun,405,0
3,하여,Verb,33,0
4,의도,Noun,18,0


In [13]:
sdf = df.sort_values("형태소")
sdf.head()

,형태소,품사,등장횟수,동일형태소
22573,,Foreign,1,0
9072,#Fall,Hashtag,1,0
9075,#Speechless,Hashtag,1,0
9070,#감정,Hashtag,1,0
20224,#꿀잼,Hashtag,1,0


In [14]:
for i in range(len(sdf)-1):
    if sdf.iloc[i,0] == sdf.iloc[i+1,0]:
        sdf.iloc[i,3] = 1
        sdf.iloc[i+1,3] = 1

In [16]:
ddf = sdf[sdf['동일형태소'] == 1]
ddf

,형태소,품사,등장횟수,동일형태소
227,ㅋ,KoreanParticle,326,1
20140,ㅋ,Foreign,1,1
1625,각,Modifier,9,1
7236,각,Noun,17,1
6473,간,Foreign,3,1
...,...,...,...,...
128,화,Suffix,394,1
914,후,Noun,88,1
11061,후,Foreign,1,1
4290,히,Adverb,62,1


In [24]:
d2df = [item for item in ddf['형태소'] if len(item) >= 2]
d2df

['같음',
 '같음',
 '개봉',
 '개봉',
 '고전',
 '고전',
 '그냥',
 '그냥',
 '그래도',
 '그래도',
 '그런',
 '그런',
 '그만',
 '그만',
 '까지',
 '까지',
 '나라',
 '나라',
 '남기고',
 '남기고',
 '남은',
 '남은',
 '남음',
 '남음',
 '내내',
 '내내',
 '넘은게',
 '넘은게',
 '놀이',
 '놀이',
 '누나',
 '누나',
 '느낌',
 '느낌',
 '는데',
 '는데',
 '다른',
 '다른',
 '다섯',
 '다섯',
 '다섯',
 '동안',
 '동안',
 '되게',
 '되게',
 '된거',
 '된거',
 '둘째',
 '둘째',
 '들어와서',
 '들어와서',
 '또한',
 '또한',
 '마다',
 '마다',
 '많이',
 '많이',
 '몇몇',
 '몇몇',
 '모두',
 '모두',
 '모든',
 '모든',
 '모름',
 '모름',
 '무슨',
 '무슨',
 '무신',
 '무신',
 '바른',
 '바른',
 '보고',
 '보고',
 '보구',
 '보구',
 '보신',
 '보신',
 '보실',
 '보실',
 '보임',
 '보임',
 '볼매',
 '볼매',
 '부터',
 '부터',
 '사이',
 '사이',
 '살림',
 '살림',
 '살면서',
 '살면서',
 '삼대',
 '삼대',
 '새벽',
 '새벽',
 '서른',
 '서른',
 '쓰는데',
 '쓰는데',
 '아름다운',
 '아름다운',
 '아무',
 '아무',
 '아아',
 '아아',
 '어느',
 '어느',
 '어떤',
 '어떤',
 '어어',
 '어어',
 '어울림',
 '어울림',
 '어쩔',
 '어쩔',
 '언니',
 '언니',
 '여러',
 '여러',
 '여섯',
 '여섯',
 '영화',
 '영화',
 '오는',
 '오는',
 '오랜',
 '오랜',
 '오빠',
 '오빠',
 '오오',
 '오오',
 '온갖',
 '온갖',
 '우우',
 '우우',
 '이나',
 '이나',
 '이런',
 '이런',
 '이만',


In [25]:
ddf[ddf['형태소'] == '다섯']

,형태소,품사,등장횟수,동일형태소
1391,다섯,Determiner,19,1
1292,다섯,Modifier,3,1
19358,다섯,Noun,2,1


In [26]:
ddf[ddf['형태소'] == '이만']

,형태소,품사,등장횟수,동일형태소
2642,이만,Noun,2,1
11368,이만,Adverb,1,1
8246,이만,Modifier,1,1
